In [ ]:
!pip install datasets # либа для установки датасета

In [2]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, TrainingArguments, TFAutoModelForSeq2SeqLM
from transformers import Trainer
from transformers import DataCollatorForSeq2Seq, AdamWeightDecay
from datasets import load_dataset
!pip install transformers[sentencepiece]

# все импорты

In [3]:
dataset = load_dataset("Helsinki-NLP/opus-100", "en-ru") # Загружем датасет

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating test split:   0%|          | 0/2000 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/1000000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [4]:
model_checkpoint = "Helsinki-NLP/opus-mt-en-ru" # Запишем название модели

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint) # Создадим токенизатор

tokenizer_config.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.38k [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/803k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/1.08M [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.60M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:197: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


In [5]:
print(tokenizer("Hello, world"))
print(tokenizer.decode(564)) # Пример декодирования

{'input_ids': [160, 5270, 2, 564, 0], 'attention_mask': [1, 1, 1, 1, 1]}
world


Предобработка и токенизация


In [6]:
max_len = 256

source_lang="en"
target_lang="ru"

def preprocess_function(examples):
  inputs = [ex[source_lang] for ex in examples["translation"]]
  targets = [ex[target_lang] for ex in examples["translation"]]
  model_inputs = tokenizer(inputs, max_length=max_len, truncation=True)

  with tokenizer.as_target_tokenizer():
    labels = tokenizer(targets, max_length=max_len, truncation=True)

  model_inputs["labels"] = labels["input_ids"]
  return model_inputs

In [8]:
tokenized_datasets = dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3892: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [9]:
model = TFAutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)
# Загрузили предобученную модельку

tf_model.h5:   0%|          | 0.00/307M [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFMarianMTModel.

All the layers of TFMarianMTModel were initialized from the model checkpoint at Helsinki-NLP/opus-mt-en-ru.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFMarianMTModel for predictions without further training.


generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

In [10]:
batch_size = 16
lr = 2e-5
weight_decay= 0.01
num_train_epoch = 1
# Гиперпараметры для модели

In [11]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, return_tensors="tf")

In [12]:
generation_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, return_tensors="tf", pad_to_multiple_of=128)

Подготовим датасеты

In [13]:
train_dataset = model.prepare_tf_dataset(
    tokenized_datasets["train"],
    batch_size=8,
    shuffle=True,
    collate_fn=data_collator
                                         )

In [14]:
validation_dataset = model.prepare_tf_dataset(
    tokenized_datasets["validation"],
    batch_size=batch_size,
    shuffle=True,
    collate_fn=data_collator
                                         )

In [15]:
generation_dataset = model.prepare_tf_dataset(
    tokenized_datasets["validation"],
    batch_size=8,
    shuffle=True,
    collate_fn=data_collator
                                         )

Компиляция, дообучение модели

In [16]:
optimizer = AdamWeightDecay(lr=lr, weight_decay_rate=weight_decay)
# Оптимизатор

/usr/local/lib/python3.10/dist-packages/tf_keras/src/optimizers/legacy/adam.py:118: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [17]:
model.compile(optimizer=optimizer)
# Компиляция модели

In [ ]:
model.fit(train_dataset, validation_data=validation_dataset, epochs=1)
# Начнем обучение

In [19]:
model.save_pretrained("tf_model/")
# Сохраним обученную модель

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 512, 'num_beams': 4, 'bad_words_ids': [[62517]], 'forced_eos_token_id': 0}


In [44]:
def translate(prompt):
  encoded = tokenizer([prompt], return_tensors="np")
  text = model.generate(**encoded, max_length=max_len)

  with tokenizer.as_target_tokenizer():
    return tokenizer.decode(text[0], skip_special_tokens=True)

Функция для взаимодействия с моделью (тоже токенизация, и позже мы получаем результат перевода)

Развернем модельку в GUI

In [ ]:
# model = TFAutoModelForSeq2SeqLM.from_pretrained("tf_train")
# Я когда скидывал дз случайно удалил модель, позже залью

In [ ]:
!pip install gradio  # Либа для UI в консоли

In [30]:
import gradio as gr

In [33]:
translate("Hello, World!") # Проверим работоспособность функции

'Привет, Мир!'

In [42]:
interface = gr.Interface(fn=translate, inputs=gr.Textbox(lines=2, placeholder="Текст для перевода"), outputs="text")
# Создадим интерфейс

In [43]:
interface.launch() # Запустим интерфейс

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://f88f43570fa9663fd0.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [45]:
import os

current_path = os.getcwd()
print(current_path)

/content


In [49]:
import shutil
from google.colab import drive

In [56]:
from google.colab import files

In [54]:
# drive.mount('/content/drive')

file_path_colab = '/content/tf_model/tf_model.h5'

file_path_drive = '/content/drive/MyDrive/tf_model.h5'

# Копирование файла с Google Colab на Google Drive
shutil.copy(file_path_colab, file_path_drive)

'/content/drive/MyDrive/tf_model.h5'

In [57]:
files.download(file_path_colab)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>